The source had to be manually scraped because the page information was generated from javascript, and I could not get Selenium working for some reason. It wasn't too much to manually scrape anyway.

In [1]:
fda = open('aggregated.txt','r')
from bs4 import BeautifulSoup
import re
import pandas
fda_read = BeautifulSoup(fda, "lxml")
print fda_read.prettify()

<html>
 <body>
  <div class="row ng-scope" id="foiDocuments" ng-if="foiDrugSummaries!= null" style="padding-bottom: 20px; padding-left: 60px; padding-right: 20px;">
   <div>
    <!-- ngRepeat: file in foiDrugSummaries -->
    <div class="col-sm-12 ng-scope colors0" id="foi" ng-class="'colors' + ($index % 2)" ng-repeat="file in foiDrugSummaries" style="border: none;">
     <br/>
     <div class="row">
      <div class="col-sm-2">
       <a class="ng-binding" href="https://animaldrugsatfda.fda.gov/adafda/app/search/public/document/downloadFoi/256" target="_blank">
        NADA
								006-707
       </a>
      </div>
      <div class="col-sm-2">
       <span>
        Approval Type:
       </span>
      </div>
      <div class="col-sm-3 ng-binding">
       Supplemental approval
      </div>
      <div class="col-sm-2">
       <span>
        Date of Approval:
       </span>
      </div>
      <div class="col-sm-3 ng-binding">
       June 02, 1997
      </div>
     </div>
     <div class="r

In [2]:
# application_numbers consists of a list of the numbers I'm interested in getting. 
# these numbers are in the aggregate.txt file. I want to use this list to find the urls associated with those numbers.

appnums = open('application_numbers.txt','r')
numlist = appnums.read()
numlist = numlist.splitlines()
appnums.close()

In [3]:
# Generate lists, populate lists if item matches. 
linklist = []
appnumlist = []
datelist = []
tablelist = fda_read.find_all(id="foi")
for row in tablelist:
    nada = row.find("a", string=re.compile(r'\b(?:%s)\b' % '|'.join(numlist)))
    if nada != None:
        appnumlist.append(row.find(string=re.compile(r'\b(?:%s)\b' % '|'.join(numlist))))
        linkid = nada.get('href')
        linklist.append(linkid)
        datelist.append(row.find(class_="col-sm-3 ng-binding").find_next(class_="col-sm-3 ng-binding").get_text(strip = True))
# Clean up application numbers
appnumlist = [a[-7:] for a in appnumlist]

In [4]:
# Create CSV by mashing all the lists into a df
df = pandas.DataFrame(appnumlist, columns =['Application Number'])
df['Link']= linklist
df['Date']= datelist
df.to_csv('appdates.csv')

In [5]:
# Generate a list of application numbers that were not found. 
fulllist = set(numlist)
foundlist = set(appnumlist)
notfound = list(fulllist.difference(foundlist))
dfnf = pandas.DataFrame(notfound, columns =['Items Not Found'])
dfnf.to_csv('appdates_notfound.csv')
# Can I generate the .csv without having to make a df first? 

hooray~ i have written my first web scraper.